# Data process and visualize using jupyter book

## Introduction
Today i will follow insturction from [Interactive Data Analysis with FigureWidget ipywidgets in Python](https://plotly.com/python/figurewidget-app/) to develop a form with HKGov's database.
---
## Importing libararies

In [1]:
import datetime
import numpy as np
import pandas as pd
import requests

import plotly.graph_objects as go
from ipywidgets import widgets

---
## Downloading material to process
We will download school list data from HKGov: [School Location and Information](https://data.gov.hk/en-data/dataset/hk-edb-schinfo-school-location-and-information) which contains he names, addresses, coordinates and other relevant information of government, private, aided, Direct Subsidy Scheme, English Schools Foundation and international schools, kindergartens and kindergartens cum-child care centre (as at 28 December 2021). For precise, the file link is: [http://www.edb.gov.hk/attachment/en/student-parents/sch-info/sch-search/sch-location-info/SCH_LOC_EDB.xlsx](http://www.edb.gov.hk/attachment/en/student-parents/sch-info/sch-search/sch-location-info/SCH_LOC_EDB.xlsx)

We will use this file as our raw data and visualize it.

In [2]:
url = 'http://www.edb.gov.hk/attachment/en/student-parents/sch-info/sch-search/sch-location-info/SCH_LOC_EDB.xlsx'
r = requests.get(url, allow_redirects=True)
open('schools.xlsx', 'wb').write(r.content)

df = pd.read_excel('schools.xlsx')
df = df.drop(df.columns[[0]], axis=1)

df.sample(3)

,ENGLISH CATEGORY,中文類別,ENGLISH NAME,中文名稱,ENGLISH ADDRESS,中文地址,LONGITUDE,經度,LATITUDE,緯度,...,SCHOOL LEVEL,學校類型,TELEPHONE,聯絡電話,FAX NUMBER,傳真號碼,WEBSITE,網頁,RELIGION,宗教
3408,Kindergartens,幼稚園,THE CHURCH OF CHRIST IN CHINA KEI CHUN KINDERG...,中華基督教會基真幼稚園,"UNIT NOS. 114-127, G/F., LAI LO HOUSE, LAI KOK...",九龍深水埗麗閣邨麗蘿樓地下１１４－１２７號,114.157012,114.157012,22.331746,22.331746,...,KINDERGARTEN,幼稚園,23618246,23618246,27487036,27487036,http://kck.ccc.edu.hk,http://kck.ccc.edu.hk,PROTESTANTISM / CHRISTIANITY,基督教
454,Aided Secondary Schools,資助中學,BUDDHIST TAI HUNG COLLEGE,佛教大雄中學,38 CHEUNG FAT STREET SO UK SHAMSHUIPO KOWLOON,九龍蘇屋深水埗長發街38號,114.158188,114.158188,22.340106,22.340106,...,SECONDARY,中學,"23862988,23876800","23862988,23876800",23869365,23869365,http://www.bthc.edu.hk,http://www.bthc.edu.hk,BUDDHISM,佛教
3088,Kindergartens,幼稚園,TAI PING KINDERGARTEN,太平幼稚園,"UNIT NO.111-115, G/F., PING YEE HOUSE, TAI PIN...",新界上水太平邨平易樓地下,114.126174,114.126174,22.497145,22.497145,...,KINDERGARTEN,幼稚園,26733199,26733199,26734561,26734561,http://www.taiping.edu.hk,http://www.taiping.edu.hk,TAOISM,道教
